<a href="https://colab.research.google.com/github/nicikess/hsg-ml-course/blob/main/notebooks/ml_challenge_ResNet_50_geo_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image

!pip install rasterio
!pip install tqdm
import rasterio
from rasterio.plot import reshape_as_image
from tqdm import tqdm

!pip install torchgeo
import torchgeo.models

from datetime import datetime

from tifffile import imread

from sklearn.metrics import confusion_matrix
from sklearn import model_selection

import torch, torchvision
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import transforms
from torchvision.transforms import ToTensor

     |████████████████████████████████| 19.3 MB 3.7 MB/s 
     |████████████████████████████████| 204 kB 4.1 MB/s 
     |████████████████████████████████| 584 kB 54.7 MB/s 
     |████████████████████████████████| 16.7 MB 74.6 MB/s 
     |████████████████████████████████| 74 kB 4.3 MB/s 
     |████████████████████████████████| 431 kB 87.2 MB/s 
     |████████████████████████████████| 6.3 MB 57.6 MB/s 
     |████████████████████████████████| 493 kB 71.0 MB/s 
     |████████████████████████████████| 1.0 MB 63.5 MB/s 
     |████████████████████████████████| 409 kB 91.0 MB/s 
     |████████████████████████████████| 88 kB 9.4 MB/s 
     |████████████████████████████████| 112 kB 71.7 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
     |████████████████████████████████| 136 kB 95.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.2 MB/s 
     |████████████████████████████████| 376 kB 70.0 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
     |██████████████

In [3]:
#Loads tiff folders

import os
folder = '/content/drive/MyDrive/ML/data/dataset/trainset/'
subfolders = []
for dirs in os.walk(folder):
  subfolders.append(dirs[0])

#remove directory
subfolders.pop(0)
for i in range(0, len(subfolders)):
     print(subfolders[i])

/content/drive/MyDrive/ML/data/dataset/trainset/AnnualCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Forest
/content/drive/MyDrive/ML/data/dataset/trainset/HerbaceousVegetation
/content/drive/MyDrive/ML/data/dataset/trainset/Highway
/content/drive/MyDrive/ML/data/dataset/trainset/Industrial
/content/drive/MyDrive/ML/data/dataset/trainset/Pasture
/content/drive/MyDrive/ML/data/dataset/trainset/PermanentCrop
/content/drive/MyDrive/ML/data/dataset/trainset/Residential
/content/drive/MyDrive/ML/data/dataset/trainset/River
/content/drive/MyDrive/ML/data/dataset/trainset/SeaLake


In [4]:
#Define paths

ROOT_PATH = '/content/drive/MyDrive/ML/'
BASE_PATH = os.path.join(ROOT_PATH, 'data')
MODEL_PATH = os.path.join(ROOT_PATH, 'model')

file_path_train = '/content/drive/MyDrive/ML/data/dataset/train.csv'
file_path_test = '/content/drive/MyDrive/ML/data/dataset/test.csv'
data_df_train = pd.read_csv(file_path_train)
data_df_test = pd.read_csv(file_path_test)

data_df_train.columns = data_df_train.columns.str.replace('path', 'image_id')
data_df_train.columns = data_df_train.columns.str.replace('Label', 'label')

In [5]:
data_df_train.head()

,image_id,label
0,AnnualCrop/AnnualCrop_135.tif,AnnualCrop
1,AnnualCrop/AnnualCrop_692.tif,AnnualCrop
2,AnnualCrop/AnnualCrop_2218.tif,AnnualCrop
3,AnnualCrop/AnnualCrop_2679.tif,AnnualCrop
4,AnnualCrop/AnnualCrop_926.tif,AnnualCrop


In [6]:
type(data_df_train)

pandas.core.frame.DataFrame

In [7]:
#Init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

In [8]:
#Classes and labels

IDX_CLASS_LABELS = {
    0: 'AnnualCrop',
    1: 'Forest', 
    2: 'HerbaceousVegetation',
    3: 'Highway',
    4: 'Industrial',
    5: 'Pasture',
    6: 'PermanentCrop',
    7: 'Residential',
    8: 'River',
    9: 'SeaLake'
}

CLASSES = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture','PermanentCrop','Residential','River', 'SeaLake']
CLASS_IDX_LABELS = dict()
for key, val in IDX_CLASS_LABELS.items():
  CLASS_IDX_LABELS[val] = key

NUM_CLASSES = len(IDX_CLASS_LABELS.items())
torch.manual_seed(10)
VALID_SIZE = 0.1

In [9]:
#Encoder and decoder

## Give idx of each class name
def encode_label(label):
    idx = CLASS_IDX_LABELS[label] 
    return idx

## Take in idx and return the class name
def decode_target(target, text_labels=True):
    result = []
    if text_labels:
        return IDX_CLASS_LABELS[target]
    else:
        return target

In [10]:
#Open tiff files with rasterio and safe to dictionary

'''import os

data = {}

for i in range(len(subfolders)):

  txtfiles = []

  # traverse whole directory
  for root, dirs, files in os.walk(subfolders[i]):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            txtfiles.append(os.path.join(root, file))

  code = subfolders[i].split('/')[8]
  index = encode_label(code)
  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")

  print(code)
  print(index)

  i = 0
  for pic in txtfiles:
    with rasterio.open(pic, "r") as img:
      arr[i] = np.moveaxis(img.read(), 0, 2)
      i += 1
      
  data[index] = arr'''

'import os\n\ndata = {}\n\nfor i in range(len(subfolders)):\n\n  txtfiles = []\n\n  # traverse whole directory\n  for root, dirs, files in os.walk(subfolders[i]):\n    # select file name\n    for file in files:\n        # check the extension of files\n        if file.endswith(\'.tif\'):\n            txtfiles.append(os.path.join(root, file))\n\n  code = subfolders[i].split(\'/\')[8]\n  index = encode_label(code)\n  arr = np.zeros([len(txtfiles), 64, 64, 13], dtype="float32")\n\n  print(code)\n  print(index)\n\n  i = 0\n  for pic in txtfiles:\n    with rasterio.open(pic, "r") as img:\n      arr[i] = np.moveaxis(img.read(), 0, 2)\n      i += 1\n      \n  data[index] = arr'

In [11]:
#Save the files as .npy

'''for i in range(len(data)):
  keys_list = list(data)
  key = keys_list[i]
  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)'''

"for i in range(len(data)):\n  keys_list = list(data)\n  key = keys_list[i]\n  np.save('/content/drive/MyDrive/ML/data/dataset/trainset/'+f'data_class{key}.npy', data[i], allow_pickle=True, fix_imports=True)"

In [12]:
#Reload the .npy files

import os
path = '/content/drive/MyDrive/ML/data/dataset/trainset/'
npyfiles = []
data = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  index = path.split('/')[8].split('_')[1][5]
  index = int(index)
  data[index] = arr

In [13]:
from pathlib import Path
class EuroSAT(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        row = self.train_df.loc[idx]
        img_id, label = row['image_id'], row['label']
        img_id = int(img_id.split('.')[0].split('_')[1])
        label_encode = int(encode_label(label))
        id = img_id - 1
        img = data[label_encode][id]
        img_rgb = img[:, :, [4,3,2]]
        if self.transform:
            img_rgb = self.transform(img_rgb)
        return img_rgb, encode_label(label)

In [14]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Resize(size=(224, 224)),
    transforms.ConvertImageDtype(torch.float),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [15]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),transforms.ConvertImageDtype(torch.float),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_ds = EuroSAT(data_df_train, BASE_PATH, transf)
print(len(train_ds))

27000


In [16]:
train_ds.transform

Compose(
    ToTensor()
    ConvertImageDtype()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [17]:
train_size = int(0.8 * len(data_df_train))
test_size = len(data_df_train) - train_size
train_ds, validation_ds = torch.utils.data.random_split(train_ds, [train_size, test_size])

In [18]:
print(len(train_ds))
print(len(validation_ds))

21600
5400


Model

In [20]:
model = torchgeo.models.resnet50("sentinel2","all",pretrained=True)

Downloading: "https://zenodo.org/record/5610000/files/resnet50-sentinel2.pt" to /root/.cache/torch/hub/checkpoints/resnet50-sentinel2.pt


  0%|          | 0.00/90.2M [00:00<?, ?B/s]

In [21]:
model

ResNet(
  (conv1): Conv2d(10, 64, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [45]:
pretrainedBool = True
#model = models.resnet152(pretrained=True)
if pretrainedBool:
    for param in model.parameters():
        param.requires_grad = False
#n_inputs = model.fc.in_features
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2), bias=False)
model.fc = nn.Sequential(
                      nn.Linear(2048, 256),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(256, NUM_CLASSES),
                      nn.LogSoftmax(dim=1)
                        )

In [ ]:
#model = models.convnext_base(pretrained=True)

In [23]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

[LOG] notebook with cpu computation enabled


In [47]:
model = model.to(device)

In [26]:
# define the optimization criterion / loss function
ce_loss = nn.CrossEntropyLoss()

In [27]:
ce_loss = ce_loss.to(device)

In [29]:
mini_batch_size = 64

In [30]:
train_dl = DataLoader(train_ds, batch_size=mini_batch_size, shuffle=True)

In [31]:
vali_dataloader = DataLoader(validation_ds, batch_size=5400, num_workers=2, pin_memory=True)

In [34]:
max_epochs_stop = 51
max_lr = 1e-3
grad_clip = 0.1
weight_decay = 0
epochs = 40
opt_func = torch.optim.Adam
milestones = [50,75,90]

train(epochs, max_lr, model, train_dl, milestones, weight_decay, grad_clip, opt_func, max_epochs_stop)

[LOG 20220512-13:02:14] epoch: 0 train-loss: 0.75438145764128
[LOG 20220512-13:03:04] epoch: 1 train-loss: 0.5942776863567928
[LOG 20220512-13:03:45] epoch: 2 train-loss: 0.5529611443979501
[LOG 20220512-13:04:25] epoch: 3 train-loss: 0.5330416139589964
[LOG 20220512-13:05:05] epoch: 4 train-loss: 0.5188070549767398
[LOG 20220512-13:05:46] epoch: 5 train-loss: 0.48979567793699413
[LOG 20220512-13:06:26] epoch: 6 train-loss: 0.4837497350203215
[LOG 20220512-13:07:06] epoch: 7 train-loss: 0.4771811928622116
[LOG 20220512-13:07:46] epoch: 8 train-loss: 0.475235055625086
[LOG 20220512-13:08:26] epoch: 9 train-loss: 0.4442080753971134
[LOG 20220512-13:09:06] epoch: 10 train-loss: 0.4523841770883848
[LOG 20220512-13:09:52] epoch: 11 train-loss: 0.44180417197519506
[LOG 20220512-13:10:33] epoch: 12 train-loss: 0.43339532388914265
[LOG 20220512-13:11:13] epoch: 13 train-loss: 0.43798665411373566
[LOG 20220512-13:11:53] epoch: 14 train-loss: 0.42702261323230506
[LOG 20220512-13:12:33] epoch: 15

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
# define learning rate and optimization strategy
#params = model.fc.parameters() if pretrainedBool else model.parameters()
#optimizer = torch.optim.Adam(params, learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

In [33]:
def train(epochs, max_lr,  model, train_loader, milestones, weight_decay=0,\
                grad_clip=None,opt_func=torch.optim.SGD, max_epochs_stop=3):
  
  # init collection of training epoch losses
  train_epoch_losses = []

  # set the model in training mode
  model.train()

  optimizer = opt_func(model.fc.parameters(), lr=max_lr, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=None, gamma=0.1)

  # train the CIFAR10 model
  for epoch in range(epochs):
      
      # init collection of mini-batch losses
      train_mini_batch_losses = []

      # iterate over all-mini batches
      for i, (images, labels) in enumerate(train_dl):

          #print(images)

          # push mini-batch data to computation device
          images = images.to(device)
          labels = labels.to(device)

          # run forward pass through the network
          output = model(images)
          
          # determine classification loss
          loss = ce_loss(output, labels)
          #loss.requires_grad = True
          
          # run backward pass
          loss.backward()
          
          # update network paramaters
          optimizer.step()

          # reset graph gradients
          optimizer.zero_grad()
          
          # collect mini-batch reconstruction loss
          train_mini_batch_losses.append(loss.data.item())

      # determine mean min-batch loss of epoch
      train_epoch_loss = np.mean(train_mini_batch_losses)

      #Added
      #result = evaluate(model, vali_dataloader)
      #scheduler.step(result['val_loss'])
      
      # print epoch loss
      now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
      print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
      
      if (epoch%10 == 0): 
          # set filename of actual model
          model_name = 'challenge_model_epoch_{}.pth'.format(str(epoch))
          # save current model to GDrive models directory
          torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name))

      # determine mean min-batch loss of epoch
      train_epoch_losses.append(train_epoch_loss)

In [35]:
import os
path = '/content/drive/MyDrive/ML/data/dataset/testset/'
npyfiles = []
datatest = {}

for root, dirs, files in os.walk(path):
    # check the extension of files
    for file in files:
        # check the extension of files
        if file.endswith('.npy'):
            npyfiles.append(os.path.join(root, file))

for i in range(len(npyfiles)):
  path = npyfiles[i]
  arr = np.load(path)
  index = int(path.split('_')[1].split('.')[0])
  datatest[index] = arr

In [36]:
from pathlib import Path
class EuroSATTest(Dataset):
    def __init__(self, train_df, train_dir, transform=None):
        self.train_dir = train_dir
        self.train_df = train_df
        self.transform = transform

    def __len__(self):
        return len(self.train_df)
    
    def __getitem__(self, idx):
        img = datatest[idx]
        img = img[:, :, [4,3,2]]
        img = np.float32(img)
        if self.transform:
            img = self.transform(img)
        return img

In [37]:
#Test data
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_ds = EuroSATTest(data_df_test, BASE_PATH, transf)
print(len(test_ds))

4232


In [38]:
resnet50_vali_dataloader = DataLoader(validation_ds, batch_size=5400, num_workers=2, pin_memory=True)

In [ ]:
resnet50_test_dataloader = DataLoader(test_ds, batch_size=10_000, num_workers=2, pin_memory=True)

In [39]:
# Display image and label.
validation_features, validation_labels = next(iter(resnet50_vali_dataloader))
print(f"Labels batch shape: {validation_labels.size()}")

Labels batch shape: torch.Size([5400])


In [44]:
# restore pre-trained model snapshot
best_model_name = os.path.join(MODEL_PATH, 'challenge_model_epoch_30.pth')

# load state_dict from path
state_dict_best = torch.load(best_model_name, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [ ]:
# set model in evaluation mode
best_model.eval()

In [ ]:
#Validation
#next() returns a tupel: tensor,label
predictions = torch.argmax(best_model(iter(resnet50_vali_dataloader).next()[0]), dim=1)

In [ ]:
#Test
predictions = torch.argmax(best_model(iter(resnet50_test_dataloader).next()), dim=1)

In [ ]:
type(predictions)

torch.Tensor

In [ ]:
list_tens = predictions.tolist()

for i in range(len(list_tens)):
  list_tens[i] = decode_target(list_tens[i])

In [ ]:
from collections import Counter
print(Counter(list_tens).keys()) # equals to list(set(words))
print(Counter(list_tens).values()) # counts the elements' frequency

In [ ]:
from sklearn import metrics
metrics.accuracy_score(validation_labels.detach().cpu(), predictions.detach().cpu())

In [ ]:
test_indices_path = '/content/drive/MyDrive/ML/data/dataset/test.csv'

final_submission_file = pd.read_csv(test_indices_path)
prediction_list = []

# turning numeric predictions to strings (as in kaggle example submission)
for p in predictions:
  prediction_list.append(decode_target(p.item()))

final_submission_file["label"] = prediction_list
### source with alternatives methods for appending to csv should this fail: https://stackoverflow.com/questions/26666919/add-column-in-dataframe-from-list
final_submission_file.to_csv('/content/drive/MyDrive/ML/data/dataset/final_submission.csv', index=None)